In [4]:

# alt_food_recommendation.py

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# -------------------- Load Dataset --------------------
df = pd.read_csv("swiggy_orders_dataset.csv")

# -------------------- Content-Based Filtering --------------------
df["Features"] = df["FoodItem"] + " " + df["Cuisine"]

vectorizer = TfidfVectorizer(stop_words="english")
tfidf_matrix = vectorizer.fit_transform(df["Features"])

cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

def content_based_recommend(food_name, num_recommendations=3):
    if food_name not in df["FoodItem"].values:
        return f"❌ '{food_name}' not found in dataset!"

    idx = df[df["FoodItem"] == food_name].index[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:num_recommendations+1]
    recommendations = [df.iloc[i[0]]["FoodItem"] for i in sim_scores]
    return recommendations

# -------------------- Collaborative Filtering (Pandas) --------------------
# Create User-Item Rating Matrix
user_item_matrix = df.pivot_table(index="UserID", columns="FoodItem", values="Rating").fillna(0)

# Compute User-User Similarity
user_sim = cosine_similarity(user_item_matrix)
user_sim_df = pd.DataFrame(user_sim, index=user_item_matrix.index, columns=user_item_matrix.index)

def collaborative_recommend(user_id, num_recommendations=3):
    if user_id not in user_item_matrix.index:
        return f"❌ User '{user_id}' not found!"

    # Find most similar users
    similar_users = user_sim_df[user_id].sort_values(ascending=False).index[1:]

    # Collect recommendations from similar users
    recs = []
    for sim_user in similar_users:
        top_foods = df[df["UserID"] == sim_user].sort_values("Rating", ascending=False)["FoodItem"].tolist()
        for food in top_foods:
            if food not in df[df["UserID"] == user_id]["FoodItem"].values:
                recs.append(food)
        if len(recs) >= num_recommendations:
            break

    return recs[:num_recommendations]

# -------------------- Hybrid Recommendation --------------------
def hybrid_recommend(user_id, food_name, num_recommendations=3):
    cb_recs = content_based_recommend(food_name, num_recommendations)
    cf_recs = collaborative_recommend(user_id, num_recommendations)
    hybrid = list(dict.fromkeys(cb_recs + cf_recs))  # remove duplicates
    return hybrid[:num_recommendations]

# -------------------- TEST --------------------
print("🍴 HYBRID FOOD RECOMMENDATION SYSTEM (No Surprise Library) 🍴")

print("\n--- Content-Based ---")
print("If you liked 'Chicken Biryani':", content_based_recommend("Chicken Biryani"))

print("\n--- Collaborative (User U002) ---")
print("Recommended for User U002:", collaborative_recommend("U002"))

print("\n--- Hybrid (User U002 + Chicken Biryani) ---")
print("Hybrid Recommendations:", hybrid_recommend("U002", "Chicken Biryani"))


🍴 HYBRID FOOD RECOMMENDATION SYSTEM (No Surprise Library) 🍴

--- Content-Based ---
If you liked 'Chicken Biryani': ['Chicken 65', 'Paneer Butter Masala', 'Veg Hakka Noodles']

--- Collaborative (User U002) ---
Recommended for User U002: ['Paneer Butter Masala', 'Margherita Pizza', 'Chicken 65']

--- Hybrid (User U002 + Chicken Biryani) ---
Hybrid Recommendations: ['Chicken 65', 'Paneer Butter Masala', 'Veg Hakka Noodles']
